In [1]:
import os
from typing import List

from langchain.agents import Tool, AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor
from langchain import hub, LLMMathChain
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser, SelfAskOutputParser, ToolsAgentOutputParser
from langchain.tools.render import render_text_description

from warnings import filterwarnings
filterwarnings("ignore", category=UserWarning)

/Users/niaki/Workspace/cocktailr-backend/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.0,
    max_tokens=None,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

In [17]:
from langchain.agents import tool

@tool
def get_most_similar_cocktail(cocktail: str) -> str:
    """Returns the most similar cocktail to the input cocktail."""
    return "Negroni"


@tool
def get_list_of_ingredients_for_a_cocktail(cocktail_name: str) -> str:
    """Returns the list of ingredients for a cocktail."""
    return "ice, ice, baby"


@tool
def get_list_of_ingredients_and_recipe_for_a_cocktail(cocktail_name: str) -> str:
    """Given a cocktail name, returns the list of ingredients for that cocktail and the recipe how to prepare it."""
    return "Ingredients: ice, ice, baby. Recipe: Mix it all together."


@tool
def get_alternatives_for_ingredient(ingredient: str) -> str:
    """Given an ingredient, returns a list of alternatives for that ingredient."""
    return "Vodka, Rum"

@tool
def get_cocktail_given_a_list_of_ingredients(ingredients: str) -> str:
    """Given ingredients, returns one cocktail that can be made from the input ingredients."""
    return "Negroni"


In [18]:
tools = [get_most_similar_cocktail, get_list_of_ingredients_and_recipe_for_a_cocktail, get_alternatives_for_ingredient, get_cocktail_given_a_list_of_ingredients]

In [19]:
agent_prompt = hub.pull("mikechan/gemini")
print(agent_prompt.template)

Your name is “Kasper”. You are world class ai personal assistant computer that created by Mike Chan(A independent AI engineer). You are highly advanced intelligent. you are very helpful and good to provide information and solve users problem. Kasper is a AELM model (Auto Execution Language Model).

“ Kasper” is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, “ Kasper” is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

“ Kasper” is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, “ Kasper” is able

In [20]:
prompt = agent_prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [21]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [22]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors="Check your output and make sure it conforms, use the Action/Action Input syntax",)

In [23]:
agent_executor.invoke({"input": "Which cocktail can I make, given that I have vodka, orange juice, and limes?"})["output"]



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_cocktail_given_a_list_of_ingredients
Action Input: vodka, orange juice, and limesNegroniDo I need to use a tool? No
Final Answer: You can make a Negroni with vodka, orange juice, and limes.

> Finished chain.


'You can make a Negroni with vodka, orange juice, and limes.'

In [19]:
def invoke_agent(input_text: str) -> str:
    return agent_executor.invoke({"input": input_text})["output"]

In [21]:
temp = invoke_agent("Which cocktail can I make, given that I only have vodka and water in my fridge")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: get_most_similar_cocktail
Action Input: vodka and water

> Finished chain.


In [22]:
temp

'Thought: Do I need to use a tool? Yes\nAction: get_most_similar_cocktail\nAction Input: vodka and water'